In [ ]:
%load_ext autoreload
%autoreload 2
%precision %.4f
import numpy as np
np.set_printoptions(precision=4, suppress=1)

import sys,os
sys.path.append(os.path.expandvars('$HELPER_SCRIPTS_DIR' ) )

from _utils_indtool import *
from IPython import get_ipython; ipython = get_ipython()

#os.path.expandvars('$HELPER_SCRIPTS_DIR')

In [ ]:


#--start_time "2023-07-15 00:00:00"
ipython.run_line_magic('run',' /p/project/icei-hbp-2020-0012/helper_scripts/indtool.py ')

In [ ]:
line_bds_per_job

In [ ]:
slice(*line_bds)

In [ ]:
!grep -noP 'import (\w+)' ../fe*.py

In [ ]:
from _utils_indtool import starttext, endstr, timeout_text

In [ ]:
outfn

In [ ]:
'/p/project/icei-hbp-2020-0012/slurmout/ML_401211_92.out'

In [ ]:
with open(outfn) as f:
    lines_all = f.readlines()

In [ ]:
def findLineInds(txtline, fnf):
    #txtline = txtline.replace('-', '\\' + '-' )
    #print(txtline)
    so = sp.getoutput('grep -n -- ' + '"' + txtline +  '" ' + fnf)#.replace('\r','\r\n')

    lns = so.split('\n')
    lineis = []
    #print(lns)
    for ln in lns:
        coli = ln.find(':')
        lni = int(ln[:coli]); 
        #print(lni, ln)
        lineis += [lni - 1]  # because they use non-python numbering starting from 1
    return lineis,lns


In [ ]:
def isTimeout(info, lines_all, maxlines_check = 100):
    for line in lines_all[info['start']:info['end']][::-1][:maxlines_check]:
        if line.find( timeout_text ) >= 0:
            return True

In [ ]:
import re
regex = re.compile(r'.*\(effctive_id = ([0-9]+)\)')
def getRsind2Info(outfn, lines_all = None):
    if lines_all is None:
        print(f'Reading {outfn}')
        with open(outfn) as f:
            lines_all = f.readlines()
    lineis_start, lines_start = findLineInds(starttext, outfn)
    lineis_end,   lines_end   = findLineInds(endstr, outfn)

    rsind2info = {}
    for lineii,(linei,line) in enumerate(zip(lineis_start,lines_start)):
        info = {'start':linei}
        info['zero_feats'] = False
        info['timeout']    = False
        if lineii >= len(lines_end):
            info['end'] = len(lines_all) - 1
            info['exit_code'] = None
            rsind2info[runstring_ind] = linei, 
            
            info['timeout'] = isTimeout(info, lines_all)
            break
            
        linei_end = lineis_end[lineii]
        info['end'] = linei_end

        #print(line)
        #print( lines_end[lineii] )
        
        #ind = line.find(' = ')
        r = re.match(regex, line)
        runstring_ind = int( r.groups()[0] )

        EC = int( lines_end[lineii][-1] )
        zf = False
        
        if EC > 0:
            zf = lines_all[linei_end - 1].startswith('ValueError: We got zero features!')
            
            #print(EC,zf)
            if not zf:
                print(lines_all[linei_end-10:linei_end])
            #break
        #rellines[lineii] = lines_all[linei:linei_end-1]
        
        info['exit_code'] = EC
        info['zero_feats'] = zf
        
        rsind2info[runstring_ind] = info
        
    return rsind2info
getRsind2Info(outfn, lines_all)

In [ ]:
fns_info_collected

In [ ]:
import gc;gc.collect()

In [ ]:
rsind2info

In [ ]:
len(lines_per_job)

In [ ]:
fns_info_collected = []
rsind2info = {}
for outfn in field_vals['outfn']:
    if outfn in fns_info_collected:
        continue
    lines_all = None
    rsind2info_cur = getRsind2Info(outfn, lines_all)
    rsind2info.update(rsind2info_cur)
    fns_info_collected += [outfn]
    for rsind, info in rsind2info.items():
        if info['exit_code'] and not info['zero_feats']:
            print(rsind)

In [ ]:
np.savez(os.path.join( os.path.expandvars('$HELPER_SCRIPTS_DIR') , 'out.npz' ),rsind2info)

In [ ]:
bads = []
for rsind, info in rsind2info.items():
    if info['exit_code']:
        print(rsind, info)
        bads += [rsind]

In [ ]:
len(bads), list(sorted(bads))

In [ ]:
field_vals['rsind']

In [ ]:
lines_all[linei_end - 1]

In [ ]:
# how to find zerofeats without running through entire stuff?

In [ ]:

print(so)

In [ ]:
time_str_sacct = f'{start_month}/{start_day}-{start_hour}:{start_minute}'
#time_str_sacct

In [ ]:
dt
dt.strftime("%Y-%m-%dT%H:%M:%S")